# GRASS Tool with Parameters

## Interface Definition for Scripts

Since GRASS tools are executable scripts (or generally programs), the interface of a GRASS tool is the command line interface of a Python script. A dedicated function _grass.script.parser_ takes care of processing the command line arguments based on the interface description specified in a Python comment with a key-value syntax defined by GRASS GIS.

The following is an example of a script which takes two parameters: name of a vector map and name of a raster map:

In [147]:
%%writefile viewscape.py
#!/usr/bin/env python

# %module
# % description: Compute viewshed and compute statistics about visible parts of sample layers
# % keyword: viewshed
# % keyword: geometry
# % keyword: statistics
# %end
# %option G_OPT_R_ELEV
# % description: Name of input elevation raster map
# % guisection: Input
# %end
# %option G_OPT_V_INPUT
# % key: points
# % guisection: Input
# %end
# %option G_OPT_R_INPUTS
# % guisection: Input
# %end
# %option G_OPT_V_OUTPUT
# % guisection: Output
# %end
# %option G_OPT_M_NPROCS
# %end


import atexit
import multiprocessing
import csv
import io
import tempfile

from pathlib import Path

import grass.script as gs


def data_to_csv(results, file_name, csv_header):
    with open(file_name, "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.DictWriter(
            csv_file, fieldnames=csv_header, extrasaction="ignore"
        )
        writer.writeheader()
        for point, result in results:
            row = {}
            for raster_name, statistics in result.items():
                for key, value in statistics.items():
                    new_key = f"{raster_name}_{key}"
                    point[new_key] = value
            row.update(point)
            writer.writerow(row)

def output_results(results, sample_rasters, output):
    with tempfile.TemporaryDirectory() as tmp_dir:
        file_name = Path(tmp_dir) / "data.txt"
        csv_header = ["cat", "east", "north"]
        database_columns = ["cat INTEGER", "east REAL", "north REAL"]
        for raster in sample_rasters:
            for value in ["mean", "stddev", "min", "max"]:
                name = f"{raster}_{value}"
                csv_header.append(name)
                database_columns.append(f"{name} REAL")
        data_to_csv(results, file_name, csv_header)
        gs.run_command(
            "v.in.ascii",
            input=file_name,
            output=output,
            format="point",
            separator="comma",
            cat=1,
            x=2,
            y=3,
            skip=1,
            columns=database_columns,
        )


def clean(name):
    gs.run_command("g.remove", type="raster", name=name, flags="f", superquiet=True)


def multiple_viewsheds(
    elevation,
    points,
    sample_rasters,
    output,
    nprocs,
):
    point_data = gs.read_command(
        "v.out.ascii",
        input=points,
        type="point",
        format="point",
        separator="comma",
        flags="cr",
    )
    reader = csv.DictReader(io.StringIO(point_data))
    data = []
    for point in reader:
        data.append((elevation, point, sample_rasters))

    with multiprocessing.Pool(processes=nprocs) as pool:
        results = pool.starmap(parametrized_viewshed, data)
    output_results(results, sample_rasters, output)


def parametrized_viewshed(elevation, point, sample_rasters):
    coordinates = (point["east"], point["north"])
    result = one_viewshed(
        elevation=elevation,
        coordinates=coordinates,
        sample_rasters=sample_rasters,
    )
    return point, result


def one_viewshed(
    elevation,
    coordinates,
    sample_rasters,
):
    viewshed = gs.append_node_pid(
        gs.legalize_vector_name(f"tmp_viewshed_{coordinates}")
    )
    atexit.register(clean, viewshed)
    gs.run_command(
        "r.viewshed",
        input=elevation,
        output=viewshed,
        coordinates=coordinates,
        flags="cb",
        quiet=True,
    )
    gs.run_command("r.null", map=viewshed, setnull=0)
    results = {}
    for name in sample_rasters:
        table_data = gs.read_command(
            "r.univar",
            map=name,
            zones=viewshed,
            quiet=True,
            flags="t",
            separator="comma",
        )
        reader = csv.DictReader(io.StringIO(table_data))
        for row in reader:
            del row["zone"]
            del row["label"]
            del row["non_null_cells"]
            del row["null_cells"]
            results[name] = row
    return results


def main():
    options, unused_flags = gs.parser()
    sample_rasters = options["input"].split(",")
    multiple_viewsheds(
        elevation=options["elevation"],
        points=options["points"],
        sample_rasters=sample_rasters,
        output=options["output"],
        nprocs=int(options["nprocs"]),
    )


if __name__ == "__main__":
    main()

Overwriting viewscape.py


As before, we will make the script executable:

In [130]:
!chmod u+x viewscape.py

Running the script with `--help` gives its interface described for command line use:

In [138]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py elevation=elevation points=firestations input=elevation,ndvi output="points" --o nprocs=4

Starting GRASS GIS...
Cleaning up temporary files...
Executing <./viewscape.py elevation=elevation points=firestations input=elevation,ndvi output=points --o nprocs=4> ...
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28   0  30   2  32   4  34   6  36   8  38  10  40  12  42  14  44  16  46  18  48  20  22  24  26  28  30  32  34  36  38  40  42  44  46  50  52  48  54  56  50  52  54  56  58  60  62  58  60  62  64  64  66  66  68  70  72  74  76  78  80  82  84  86  88  68  90  70  92  72  94  74  96  76  98  78 100
  80  82  84  86  88  90  92  94  96  98 100
Scanning input for column types...
Number of columns: 11
Number of data rows: 3
Importing points...
   0  25  50  75 100
Populating table...
Building topology for vector map <points@foss4g>...
Registering primiti

In [150]:
!black --diff viewscape.py
!black viewscape.py
!flake8 --max-line-length=88 --ignore=E501 viewscape.py
!pylint --disable=E0401,C0114,C0116 viewscape.py

--- viewscape.py	2023-10-20 20:09:16.971428 +0000
+++ viewscape.py	2023-10-20 20:09:58.571659 +0000
@@ -35,22 +35,21 @@
 import grass.script as gs
 
 
 def data_to_csv(results, file_name, csv_header):
     with open(file_name, "w", newline="", encoding="utf-8") as csv_file:
-        writer = csv.DictWriter(
-            csv_file, fieldnames=csv_header, extrasaction="ignore"
-        )
+        writer = csv.DictWriter(csv_file, fieldnames=csv_header, extrasaction="ignore")
         writer.writeheader()
         for point, result in results:
             row = {}
             for raster_name, statistics in result.items():
                 for key, value in statistics.items():
                     new_key = f"{raster_name}_{key}"
                     point[new_key] = value
             row.update(point)
             writer.writerow(row)
+
 
 def output_results(results, sample_rasters, output):
     with tempfile.TemporaryDirectory() as tmp_dir:
         file_name = Path(tmp_dir) / "data.t

# Points


In [66]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec g.region raster=elevation
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec v.out.ascii input=firestations type="point" format="point" separator="comma" -c -r

Starting GRASS GIS...
Cleaning up temporary files...
Executing <g.region raster=elevation> ...
Execution of <g.region raster=elevation> finished.
Cleaning up temporary files...
Starting GRASS GIS...
Cleaning up temporary files...
Executing <v.out.ascii input=firestations type=point format=point separator=comma -c -r> ...
east,north,cat
630420.50034711,215694.02807559,9
630879.21198056,224876.55413017,12
635775.56533925,228121.69258378,19
635940.26230542,225912.79645818,20
637386.83129147,222569.15159736,21
641437.43097947,221737.70843723,22
644588.46729229,217450.61375119,23
644598.62057612,221014.97865205,24
642868.29195637,225195.28915104,25
640173.91152612,226162.37987726,26
640711.47282426,228461.46577181,27
644601.8806259,226660.12097034,28
633178.15477,221353.03723179,52
641803.17360861,224986.14236578,56
642559.95242121,215531.80353493,57
Execution of <v.out.ascii input=firestations type=point format=point separator=comma -c -r> finished.
Cleaning up temporary files...
